In [1]:
import pandas as pd
import numpy as np
import googlemaps
from datetime import datetime
import acquire_r
from env import google_key, google_key1
from googlemaps import convert
import wrangle_zillow_r
import wrangle_zillow
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import scipy.stats as stats
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import gmaps
import gmaps.datasets
gmaps.configure(api_key=google_key1) # Fill in with your API key
%matplotlib inline
import zillow_fe
import feature_selection
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.preprocessing import QuantileTransformer, quantile_transform
from sklearn.metrics import median_absolute_error, r2_score, mean_absolute_error
from math import sqrt
from sklearn.preprocessing import PolynomialFeatures
from pygam import LogisticGAM, LinearGAM, s, f, te
from sklearn.linear_model import RidgeCV
from sklearn.compose import TransformedTargetRegressor
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, explained_variance_score
from scipy import stats
import evaluate
import explore
import model
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [41]:
# Read CSV file which was saved after data wrangling
zillow = pd.read_csv('zillow_reduced.csv')
orig_rows = zillow.shape[0]
orig_cols = zillow.shape[1]
print (f'There are {orig_rows} rows and {orig_cols} columns in the zillow dataframe.')

There are 64786 rows and 38 columns in the zillow dataframe.


In [42]:
zillow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64786 entries, 0 to 64785
Data columns (total 38 columns):
Unnamed: 0                    64786 non-null int64
Unnamed: 0.1                  64786 non-null float64
airconditioningtypeid         64786 non-null int64
basementsqft                  64786 non-null int64
bathroomcnt                   64786 non-null int64
bedroomcnt                    64786 non-null int64
buildingqualitytypeid         64786 non-null int64
decktypeid                    64786 non-null int64
finishedsquarefeet12          64786 non-null int64
fireplacecnt                  64786 non-null int64
garagecarcnt                  64786 non-null int64
garagetotalsqft               64786 non-null int64
hashottuborspa                64786 non-null int64
heatingorsystemtypeid         64786 non-null int64
latitude                      64786 non-null float64
longitude                     64786 non-null float64
lotsizesquarefeet             64786 non-null int64
poolcnt           

In [43]:
#fix median_income coversion from object to float
zillow = wrangle_zillow_r.fix_med(zillow)
zillow['living_area'] = zillow.lotsizesquarefeet/zillow.finishedsquarefeet12

In [44]:
# split data in train/test (80/20)
train, test = wrangle_zillow.split_my_data(zillow, 0.8)

In [45]:
train.shape, test.shape

((51828, 39), (12958, 39))

In [46]:
# Split the train into train and validate sets (80/20)
train, validate = wrangle_zillow.split_my_data(train, 0.8)

In [47]:
#check train and validate shapes
train.shape, validate.shape

((41462, 39), (10366, 39))

In [48]:
#add onehot encoding if needed
#function in model.py
# add age as a variable

# Modeling

In [49]:
# Lets first establish a baseline

train["base_logerror"] = train.logerror.mean()

RMSE_base = sqrt(mean_squared_error(train.logerror,train.base_logerror))

print(f'The Baseline RMSE logerror is :{RMSE_base}')

The Baseline RMSE logerror is :0.1617462839126745


In [53]:
# Run a polynomial regression model
features = ['bathroomcnt', 'bedroomcnt', 'finishedsquarefeet12', 'taxvaluedollarcnt', 'median_income']
RMSE, R2 = model.poly_regression(train, features,5)
print(f' The RMSE is : {RMSE} and R2 is {R2}')

 The RMSE is : 0.161 and R2 is 0.012


In [54]:
# Run a polynomial regression model
features = ['bathroomcnt', 'bedroomcnt', 'finishedsquarefeet12', 'taxvaluedollarcnt', 'median_income']
RMSE, R2 = model.poly_regression(validate, features,5)
print(f' The RMSE is : {RMSE} and R2 is {R2}')

 The RMSE is : 0.16 and R2 is 0.036


In [55]:
# Run a polynomial regression model
features = ['bathroomcnt', 'bedroomcnt', 'finishedsquarefeet12', 'taxvaluedollarcnt', 'median_income']
RMSE, R2 = model.poly_regression(train, features,5)
print(f' The RMSE is : {RMSE} and R2 is {R2}')

 The RMSE is : 0.161 and R2 is 0.012


In [58]:
features = ['bathroomcnt', 'bedroomcnt', 'finishedsquarefeet12', 'taxvaluedollarcnt', 'median_income', 'living_area']
RMSE, R2 = model.poly_regression(train, features,6)
print(f' The RMSE is : {RMSE} and R2 is {R2}')

 The RMSE is : 0.159 and R2 is 0.037


In [59]:
features = ['bathroomcnt', 'bedroomcnt', 'finishedsquarefeet12', 'taxvaluedollarcnt', 'median_income', 'living_area']
RMSE, R2 = model.poly_regression(validate, features,6)
print(f' The RMSE is : {RMSE} and R2 is {R2}')

 The RMSE is : 0.154 and R2 is 0.098


In [62]:
features = ['bathroomcnt', 'bedroomcnt', 'finishedsquarefeet12', 'taxvaluedollarcnt', 'median_income', 'living_area']
RMSE, R2 = model.poly_regression(test, features,6)
print(f' The RMSE is : {RMSE} and R2 is {R2}')

 The RMSE is : 0.162 and R2 is 0.094
